In [73]:
import pandas as pd
from docplex.mp.model import Model

In [74]:
excel=pd.ExcelFile("C:/Users/Roberto/Documents/UCN/VII Semestre/Cadena de Suministros/Clases/4 - Networks Optimization/Transporte_ejemplo.xlsx")
costos=pd.read_excel(excel,"Costos")
demanda=pd.read_excel(excel,"Demanda")
produccion=pd.read_excel(excel,"Produccion")
plantas=[i for i in range(len(produccion))]
destinos=[j for j in plantas]
arcos=[(i,j) for i in plantas for j in destinos]
Demanda={(j):row["Demanda de Containers"] for j,row in demanda.iterrows()}
Capacidad={(i):row["Capacidad de Produccion"] for i,row in produccion.iterrows()}
demanda_total=sum(Demanda[i] for i in Demanda)
Costo={arcos[k]: row["Costo de Envio"] for k,row in costos.iterrows()}
print(arcos[0]) #forma de referir el valor de una LISTA de 2 subindices
print(Capacidad[0]) #forma de referir el valor de un DICCIONARIO con 1 subíndice
print(Costo[0,0]) #forma de referir el valor de un DICCIONARIO con 2 subíndices

(0, 0)
10000
1.04


In [75]:
m=Model("Transporte")

In [76]:
x=m.integer_var_dict(arcos,name="x")

In [77]:
m.minimize(m.sum(x[i,j]*Costo[i,j] for i,j in arcos))

In [78]:
m.add_constraints(m.sum(x[i,j] for j in destinos)<=Capacidad[i] for i in plantas)
m.add_constraints(m.sum(x[i,j] for i in plantas)>=Demanda[j] for j in destinos)

[docplex.mp.LinearConstraint[](x_0_0+x_1_0+x_2_0,GE,11000),
 docplex.mp.LinearConstraint[](x_0_1+x_1_1+x_2_1,GE,6300),
 docplex.mp.LinearConstraint[](x_0_2+x_1_2+x_2_2,GE,7400)]

In [79]:
print(m.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Transporte

Minimize
 obj: 1.040000000000 x_0_0 + 1.270000000000 x_0_1 + 1.220000000000 x_0_2
      + 1.230000000000 x_1_0 + 1.930000000000 x_1_1 + 0.600000000000 x_1_2
      + 1.920000000000 x_2_0 + 0.940000000000 x_2_1 + 1.030000000000 x_2_2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 <= 10000
 c2: x_1_0 + x_1_1 + x_1_2 <= 10000
 c3: x_2_0 + x_2_1 + x_2_2 <= 10000
 c4: x_0_0 + x_1_0 + x_2_0 >= 11000
 c5: x_0_1 + x_1_1 + x_2_1 >= 6300
 c6: x_0_2 + x_1_2 + x_2_2 >= 7400

Bounds

Generals
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2
End



In [80]:
sol=m.solve(log_output=True)

CPXPARAM_Read_DataCheck                          1
Found incumbent of value 30250.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 6 rows, 9 columns, and 18 nonzeros.
Reduced MIP has 0 binaries, 9 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 6 rows, 9 columns, and 18 nonzeros.
Reduced MIP has 0 binaries, 9 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.01 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                        30250.0000        0.0000           100.00%
*     0     0      integral     0    21992.0000    21992.0000        3    0.00%
Elapsed time = 0.06 sec.

In [81]:
m.get_solve_status()

<JobSolveStatus.OPTIMAL_SOLUTION: 2>

In [82]:
sol.display()

solution for: Transporte
objective: 21992.000
x_0_0 = 10000
x_1_0 = 1000
x_1_2 = 7400
x_2_1 = 6300


In [83]:
arcos_activos={(k):x[k].solution_value for k in arcos if x[k].solution_value>0}
print(arcos_activos)

{(0, 0): 10000.0, (1, 0): 1000.0, (1, 2): 7400.0, (2, 1): 6300.0}
